In [3]:
!pip install alibi
!pip install alibi-detect

  Using cached alibi-0.6.2-py3-none-any.whl (390 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached scikit_image-0.18.3-cp37-cp37m-win_amd64.whl (12.1 MB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\MasterDegree\\AdvancedTopicsInSoftwareSystems\\ProjectWork\\venv37\\Lib\\site-packages\\skimage\\data\\astronaut.png'
Check the permissions.

You should consider upgrading via the 'D:\MasterDegree\AdvancedTopicsInSoftwareSystems\ProjectWork\venv37\Scripts\python.exe -m pip install --upgrade pip' command.
ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\MasterDegree\\AdvancedTopicsInSoftwareSystems\\ProjectWork\\venv37\\Lib\\site-packages\\skimage\\io\\tests\\test_mpl_imshow.py'
Check the permissions.

You should consider upgrading via the 'D:\MasterDegree\AdvancedTopicsInSoftwareSystems\ProjectWork\venv37\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached alibi-0.6.2-py3-none-any.whl (390 kB)
  Using cached spacy-3.2.0-cp37-cp37m-win_amd64.whl (11.9 MB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)
  Using cached scikit_image-0.18.3-cp37-cp37m-win_amd64.whl (12.1 MB)
  Using cached tensorflow-2.6.2-cp37-cp37m-win_amd64.whl (423.3 MB)
  Using cached imageio-2.12.0-py3-none-any.whl (3.3 MB)
  Using cached tifffile-2021.11.2-py3-none-any.whl (178 kB)
  Using cached murmurhash-1.0.6-cp37-cp37m-win_amd64.whl (20 kB)
  Using cached blis-0.7.5-cp37-cp37m-win_amd64.whl (6.5 MB)
  Using cached srsly-2.4.2-cp37-cp37m-win_amd64.whl (450 kB)
  Using cached catalogue-2.0.6-py3-none-any.whl (17 kB)
  Using cached preshed-3.0.6-cp37-cp37m-win_amd64.whl (108 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached spacy_legacy-3.0.8-py2.py3-none-any.whl (14 kB)
  Using cached spacy_loggers-1.0.1-py3-none-any.whl (7.0 kB)
  Using cached thinc-8.0.13-c

You should consider upgrading via the 'D:\MasterDegree\AdvancedTopicsInSoftwareSystems\ProjectWork\venv37\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
import alibi
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.utils.saving import save_detector, load_detector

In [9]:
# Visualizations of distributions of features between reference and test set
def plot_categories(idx: int) -> None:
    # reference data
    x_ref_count = {f: [(X_ref[:, f] == v).sum() for v in vals]
                   for f, vals in cd.x_ref_categories.items()}
    fref_drift = {cat: x_ref_count[idx][i] for i, cat in enumerate(category_map[idx])}

    # test set
    cats = {f: list(np.unique(X_t1[:, f])) for f in categories_per_feature.keys()}
    X_count = {f: [(X_t1[:, f] == v).sum() for v in vals] for f, vals in cats.items()}
    fxt1_drift = {cat: X_count[idx][i] for i, cat in enumerate(category_map[idx])}

    # plot bar chart
    plot_labels = list(fxt1_drift.keys())
    ind = np.arange(len(plot_labels))
    width = .35
    fig, ax = plt.subplots()
    p1 = ax.bar(ind, list(fref_drift.values()), width)
    p2 = ax.bar(ind + width, list(fxt1_drift.values()), width)
    ax.set_title(f'Counts per category for {feature_names[idx]} feature')
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(plot_labels)
    ax.legend((p1[0], p2[0]), ('Reference', 'Test'), loc='upper right', ncol=2)
    ax.set_ylabel('Counts')
    ax.set_xlabel('Categories')
    plt.xticks(list(np.arange(len(plot_labels))), plot_labels, rotation='vertical')
    plt.show()

In [10]:
wine_data = pd.read_csv('./raw_data/wine_data/winequalityN.csv')
wine_data = wine_data.dropna()
wine_data.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [11]:
feature_names = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
    "quality",
]

feature_names_no_quality = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]

In [12]:
X = wine_data[feature_names_no_quality]

In [13]:
n_ref = 1000
n_test = 1000

X_ref, X_t0, X_t1 = X[:n_ref], X[n_ref:n_ref + n_test], X[n_ref + n_test:n_ref + 2 * n_test]
X_ref.shape, X_t0.shape, X_t1.shape

((1000, 11), (1000, 11), (1000, 11))

In [14]:
X_t2, X_t3, X_t4, X_t5 = X[n_ref+n_test*2:n_ref+3*n_test], X[n_ref+n_test*3:n_ref+4*n_test], X[n_ref+n_test*4:n_ref+5*n_test], X[n_ref+n_test*5::]

In [15]:
# Set the categorical mapping
categories_per_feature = {f: None for f in list(category_map.keys())}

NameError: name 'category_map' is not defined

In [16]:
# Initialize the drift detector
# cd = TabularDrift(X_ref, p_val=.05, categories_per_feature=categories_per_feature) # with categorical columns
cd = TabularDrift(X_ref, p_val=.05) # without categorical columns

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
# Save/load the drift detector
filepath = './alibi_drift_report'  # change to directory where detector is saved
save_detector(cd, filepath)
cd = load_detector(filepath)

In [ ]:
# Check the test set if drifted from reference data
preds = cd.predict(X_t0)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds['data']['is_drift']]))

In [ ]:
for f in range(cd.n_features):
    # stat = 'Chi2' if f in list(categories_per_feature.keys()) else 'K-S'
    stat = 'K-S'
    fname = feature_names_no_quality[f]
    stat_val, p_val = preds['data']['distance'][f], preds['data']['p_val'][f]
    print(f'{fname}\t--\t{stat} {stat_val:.3f}\t--\tp-value {p_val:.3f}')

In [ ]:
preds['data']['threshold']

# Online Drfit Detection for Wine Quality Dataset

In [ ]:
# Assume that the data arrive sequentially and we would like to detect drift as soon as possible
# This is achieved by compute test for every N time-steps or N samples arrived since the last test
# Being sensitive to slight or severe drift is DIFFICULT
# If the N samples (windows) is too small then small/slight drift is undetecable, while large samples hinder responsiveness to severe drift

# Soluton? => Perform a test each time data arrive
# Offline method (KS test) is not applicable since computing p-values is too expensive